In [1]:
import csv
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import tensorflow
from tensorflow import keras
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy import stats 
import os

# from sklearn.feature_selection import SelectKBest,chi2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
def cnn1model(inshape):
    cnn1 = Sequential()
    cnn1.add(Conv1D(filters=128, kernel_size=3,activation = 'relu',padding='same',input_shape=(inshape,1)))
    cnn1.add(MaxPooling1D(pool_size=2))
    cnn1.add(Dropout(0.25))

    cnn1.add(Conv1D(filters=256, kernel_size=3,padding='same',activation = 'relu'))
    cnn1.add(MaxPooling1D(pool_size=2))
    cnn1.add(Dropout(0.25))

    cnn1.add(Conv1D(filters=128, kernel_size=3,padding='same',activation = 'relu'))
    cnn1.add(MaxPooling1D(pool_size=2))
    cnn1.add(Dropout(0.25))

    cnn1.add(Flatten())
    cnn1.add(Dense(32,activation='relu'))
    cnn1.add(Dropout(0.25))
    cnn1.add(Dense(2,activation = 'softmax'))
    cnn1.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.Adam(),
            metrics=['accuracy'])
    
    return cnn1   

In [3]:
def trainmodel(X_train,y_train,X_test,y_test):
    model = None
    k = X_train.shape[1]
    model = cnn1model(k)
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=pat, mode='min', verbose=0)
    model_checkpoint = ModelCheckpoint('./data/CDHIT/kmer.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
    
    history = model.fit(X_train,y_train,
                       batch_size=32,
                       epochs =30,
                       verbose=0,
                       callbacks=[early_stopping,model_checkpoint],
                       validation_data = (X_test,y_test)
                )
    score = model.evaluate(X_test, y_test, batch_size=32, verbose=0)
    
  
    return score

In [4]:
from collections import defaultdict
from math import sqrt
def evaluate(cm):
    idx = [0,1]
    columns = ['Sn','Sp','Preci','MCC','F1-score','OA']
    eva = pd.DataFrame(index=idx,columns=columns)
    
    oa = np.diag(cm).sum()/cm.sum()
    
    for i in range(len(cm)):
        fp = cm.sum(axis=0)[i]-cm[i][i] # sum of column
        fn = cm.sum(axis=1)[i]-cm[i][i] # sum of row
        tp = cm[i][i]
        tn = cm.sum()-(fp+fn+tp)
        if tp == 0:
            sensitivity = 0
            precision = 0
        else:
            sensitivity = tp / (tp+fn)
            precision = tp / (tp+fp)
            
        specificity = tn / (tn+fp)
               
        if precision*sensitivity==0:
            f1_score=0
        else:
            f1_score = 2 * precision * sensitivity / (precision + sensitivity)
        if tp*tn-fp*fn == 0:
            mcc = 0
        else:
            mcc = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
        if i==0:
            eva.loc[i] = [sensitivity,specificity,precision,mcc,f1_score,oa]
        else:
            eva.loc[i] = [sensitivity,specificity,precision,mcc,f1_score,0]
    
    return oa , eva

In [5]:
def loaddata_cnn(df):
    
#     df.iloc[:,0:-2] = (df.iloc[:,0:-2]).div(df['length'],axis=0)

    X = df.iloc[:,1:-3]
    y = df['Class']
     
    
    y = keras.utils.to_categorical(y, num_classes=2, dtype='int')
    
    return X, y

## Load data

In [12]:
df_train = pd.read_csv("./data/CDHIT/A549train_4mer0miss.csv")
mean = df_train["A549"].mean()
std = np.std(df_train["A549"]) 
for i in range(len(df_train)):
    if df_train.loc[i,"A549"] >= mean + std:
        df_train.loc[i,"Class"] = 1
    elif df_train.loc[i,"A549"] <= mean - std:
        df_train.loc[i,"Class"] = 0
    else:
        df_train.loc[i,"Class"] = 2
train_bi_class = df_train[df_train["Class"] != 2].reset_index(drop=True)

X_train,y_train = loaddata_cnn(train_bi_class)
df_train.tail(2)

transcript_id  AAAA  AAAG  AAAT  AAAC  AAGA  AAGG  AAGT  AAGC  \
13347  ENST00000703168.1  26.0  18.0  24.0  11.0  21.0  20.0  13.0  16.0   
13348  ENST00000491586.5  26.0  17.0  24.0  11.0  19.0  20.0  13.0  16.0   

       AATA  ...  CCTG  CCTT  CCTC  CCCA  CCCG  CCCT  CCCC     A549  length  \
13347  11.0  ...  41.0  21.0  22.0  41.0   6.0  27.0  21.0 -1.28524    4254   
13348  11.0  ...  41.0  22.0  22.0  39.0   6.0  28.0  21.0 -1.28524    4101   

       Class  
13347    0.0  
13348    0.0  

[2 rows x 260 columns]

In [20]:
df_test = pd.read_csv("./data/CDHIT/A549test_4mer0miss.csv")
mean = df_test["A549"].mean()
std = np.std(df_test["A549"]) 
for i in range(len(df_test)):
    if df_test.loc[i,"A549"] >= mean + std:
        df_test.loc[i,"Class"] = 1
    elif df_test.loc[i,"A549"] <= mean - std:
        df_test.loc[i,"Class"] = 0
    else:
        df_test.loc[i,"Class"] = 2
test_bi_class = df_test[df_test["Class"] != 2].reset_index(drop=True)

# X_test,y_test = loaddata_cnn(test_bi_class)
df_test.tail(2)

transcript_id  AAAA  AAAG  AAAT  AAAC  AAGA  AAGG  AAGT  AAGC  AATA  \
3294  ENST00000703202.1   1.0   1.0   1.0   2.0   4.0   7.0   4.0   3.0   0.0   
3295  ENST00000431933.5   1.0   1.0   0.0   2.0   4.0   4.0   3.0   4.0   0.0   

      ...  CCTG  CCTT  CCTC  CCCA  CCCG  CCCT  CCCC      A549  length  Class  
3294  ...  10.0   1.0   4.0   3.0   3.0   0.0   0.0 -0.019093     858    2.0  
3295  ...   4.0   2.0   2.0   1.0   3.0   1.0   0.0 -0.019093     510    2.0  

[2 rows x 260 columns]

In [13]:
len(df_test),len(test_bi_class),len(df_train),len(train_bi_class)

(3296, 1044, 13349, 4063)

In [21]:
num = 10
scaler = StandardScaler()
acc = defaultdict()
val = defaultdict()

for i in range(num):
        X_test,y_test = loaddata_cnn(test_bi_class)
         
        train_x, val_x, train_y,  val_y= train_test_split(X_train, y_train, test_size=0.2,      
                                                              random_state=i,stratify=y_train)
        
        train_x = scaler.fit_transform(train_x)
        val_x = scaler.transform(val_x)
        X_test = scaler.transform(X_test)
        
        
        ## reshape to fit model
        k = train_x.shape[1]        
        train_x = train_x.reshape(len(train_x),k,1)
        val_x = val_x.reshape(len(val_x),k,1)  
        X_test = X_test.reshape(len(X_test),k,1)

        ## Train model
        model_history = trainmodel(train_x, train_y, val_x, val_y)
        
        model = load_model('./data/CDHIT/kmer.h5')
        val[i],acc[i] = model.evaluate(X_test,y_test)
        predict = model.predict(X_test)
        cm = confusion_matrix(y_test.argmax(axis=1), predict.argmax(axis=1))
        oa, eva = evaluate(cm)

    #         val[i],acc[i],rep,cm = trainmodel(X_train,y_train,X_test, y_test)
    #         _, _, _, _,_,_, EVA = evaluate(cm)
    #         rep.to_excel(writer,sheet_name='report'+str(i))
        cm = pd.DataFrame(cm)
        
acc_val = list(acc.values())    
avg = np.mean(acc_val)

val_loss = list(val.values())
avg1 = np.mean(val_loss)
std = np.std(acc_val,ddof=1)

print("Average accuracy of {} is {}.".format("4MER",avg))
print("The standard devation =mer is {}.".format(std))    

33/33 [==============================] - 0s 6ms/step - loss: 0.3323 - accuracy: 0.8678
Average accuracy of 4MER is 0.8746168613433838.
The standard devation =mer is 0.006627763257194829.
